# Team 7 - Garbage DumPY
## Demonstration of Basic Forward and Inverse Kinematics
### MANE 4120/6120 - Robotics 1 - Fall 2023

1. Import RPI Robotics Toolbox, Yahboom Jetson DOFBOT Code, etc.

In [8]:
import general_robotics_toolbox.general_robotics_toolbox as grt
import general_robotics_toolbox.general_robotics_toolbox_invkin as grt_ik
import Arm, Dofbot
from Arm_Lib import Arm_Device

import time
import numpy as np
import math
import path_generation
import move_read

2. Initialize Robot Object

In [9]:
# Define all the joint lengths [m]
l0 = 61 * 10**-3
l1 = 43.5 * 10**-3
l2 = 82.85 * 10**-3
l3 = 82.85 * 10**-3
l4 = 73.85 * 10**-3
l5 = 54.57 * 10**-3


# define the unit vectors
ex = np.array([1, 0, 0])
ey = np.array([0, 1, 0])
ez = np.array([0, 0, 1])


# Define the position vectors from i-1 -> i
P01 = (l0 + l1) * ez
P12 = np.zeros(3)
P23 = l2 * ex
P34 = -1*l3 * ez
P45 = np.zeros(3)
P5T = -1*(l4 + l5) * ex

# define the class inputs: rotation axes (H), position vectors (P), and joint_type
H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
P = np.array([P01, P12, P23, P34, P45, P5T]).T
joint_type = [0,0,0,0,0]

 
# define the Robot object
robot = grt.Robot(H, P, joint_type)

# initialize servo arm
Arm = Arm_Device()
# Set the bot to be vertical
Arm.Arm_serial_servo_write6(90,65,5,70,90,15,1000)
start = np.array([90,90,90,90,90,15])
# THE SERVO ANGLE (the 6th variable) CAN NOT BE ZERO OR IT WILL BREAK THE SERVO
# It must be between 5 and 180

3. Perform Forward Kinematics

In [6]:
# initialize joint angles, radians
q0 = np.array([90,65, 5,70,90])*math.pi/180
print("Initial Joints: \n",q0)
# and compute path
p = grt.fwdkin(robot,q0)
print("Expected Transform:\n",p)

Initial Joints: 
 [1.57079633 1.13446401 0.08726646 1.22173048 1.57079633]
Expected Transform:
 R = [[-0.     -0.     -1.    ]
     [-0.766   0.6428  0.    ]
     [ 0.6428  0.766  -0.    ]]
p = [0.     0.2112 0.0687]




In [7]:
#End position from FK
end = np.array([90,65,5,70,90,15])

#Generating path
q_des = path_generation.gen_joint_space_path(start, end, 5)

#Moving arm through path
for i in range(len(q_des)):
    move_read.move_bot(q_des[i])
print(q_des)

[array([90., 90., 90., 90., 90., 15.]), array([90., 85., 73., 86., 90., 15.]), array([90., 80., 56., 82., 90., 15.]), array([90., 75., 39., 78., 90., 15.]), array([90., 70., 22., 74., 90., 15.]), array([90., 65.,  5., 70., 90., 15.])]


In [3]:
q_des = path_generation.gen_joint_space_path(start, end, 5)
print(q_des)

NameError: name 'start' is not defined

In [5]:
#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

4. Perform Inverse Kinematics

In [10]:
# desired rotation matrix
Rf = np.array([[0,0,-1],[0,-1,0],[-1,0,0]])
# desired position matrix
Pf = [0.0,0,0.398]
# make desired transformation
Tf = grt.Transform(Rf,Pf)
# and calculate inverse kinematics
max_steps=200
qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
while(qf[0]!=True): #if the inverse kinematics does not converge
    max_steps*=2
    qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
print("Necessary Joint Angles:\n",np.array(qf[1][0]))

KeyboardInterrupt: 

In [9]:
# send these joint angles to servo to have the robot move to desired position
desired_joints_degrees = np.array(qf[1][0])*180/math.pi
Arm.Arm_serial_servo_write6(desired_joints_degrees[0], desired_joints_degrees[1], desired_joints_degrees[2], \
                            desired_joints_degrees[3], desired_joints_degrees[4], 155, 1100)

In [18]:
#Path Following
#ath_done = false
start = np.array([90,90,90,90,90,15])
end = np.array([90,65,22,70,90,15])

kp = .5

q_des = path_generation.gen_joint_space_path(start, end, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
    
    
    
    
        
        

    
    
    
    

[ 0.  0. -1.  0.  1. -2.]
hi
hi
hi
hi
[ 0.  0.  0.  0.  0. -1.]
[ 0.   -0.25  0.   -2.    0.   -1.  ]
hi
hi
[ 0.   -0.25  0.    0.    0.   -1.  ]
[ 0.   0.5  0.  -1.   0.  -2. ]
hi
[ 0.   0.5  0.  -1.   0.  -1. ]
[ 0.   -0.75  0.   -1.    0.   -2.  ]
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
[ 0.    0.25  0.    0.    0.   -1.  ]
[ 0. -1.  0. -1.  0. -2.]
hi
[ 0. -1.  0. -1.  0. -1.]
[ 0.   -0.25  0.   -1.    0.   -1.  ]
[ 0.   -0.25  0.   -1.    0.   -1.  ]
